In [1]:
pip install nltk

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import numpy as np
import nltk
nltk.download('vader_lexicon')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger') 
nltk.download('punkt')
nltk.download('wordnet')
from nltk.sentiment.vader import SentimentIntensityAnalyzer

sid = SentimentIntensityAnalyzer()


[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/jupyterlab/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/jupyterlab/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jupyterlab/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to /home/jupyterlab/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /home/jupyterlab/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [3]:
df = pd.read_csv('opinion_survey.csv')
df = df.replace(np.nan, 'Neutral', regex=True) ### Some values are missing, use "Neutral" for missing values
df = df.drop(df.columns[[0]],axis=1) ### Drop extra index
test_df = df.iloc[0:, 0:] # Just for renaming purposes so that df variable can be used for testing
test_df.head()

,I can easily log-in and log-out my Canvas account. (Please write your views relating your answer above.),I can access my course using the mobile Canvas app. (Please write your views about this relating to your answer above.),I can access my Canvas account using the available computers in JRU. (Please write your views about this relating to your answer above.),I have no issues accessing my Canvas account using the wired internet connection in the computer laboratories. (Please write your views about this relating to your answer above.),I can easily get help when I have problems with my Canvas account. (Please write your views about this relating to your answer above.),I can access my Canvas account outside JRU. (Please write your views about this relating to your answer above.),I can clearly understand the lessons on my Canvas modules. (Please write your views about this relating to your answer above.),I recognize that the modules in my course are well organized. (Please write your views about this relating to your answer above.),I can easily manage the navigation of the courses through the desktop version. .1,I can easily manage the navigation of the courses through the mobile app. (Please write your views about this relating to your answer above.),...,I understand the lessons better with the help of course materials in Canvas. (Please write your views about this relating to your answer above.),Canvas course materials make me interested. (Please write your views about this relating to your answer above.),"The course materials on Canvas helped me prepare for: Face to face discussions, quizzes, major exams, assignments, forums, etc. (Please write your views about this relating to your answer above.)",My instructor knows how to use Canvas. (Please write your views about this relating to your answer above.),My instructor demonstrated mastery of the subject matter. (Please write your views about this relating to your answer above.),My instructor encourages me to use Canvas. (Please write your views about this relating to your answer above.),"Canvas has made it easy for me to: communicate with my instructor, receive class announcements, share ideas on group discussions","I can answer the assessment materials (quiz, exam) posted in Canvas. (Please write your views about this relating to your answer above.)","I find the course materials on Canvas aligned with the: quizzes, major exams, assignments, forums, etc.) (Please write your views about this relating to your answer above.)",I have been give enough time to accomplish the activities on Canvas. (Please write your views about this relating to your answer above.)
0,i can,i can,i can,i can,i can,i can,i can,i can,i can,i can,...,i can,i can,i can,i can,i can,i can,i can,i can,i can,-
1,i have no problems in log in,i have no problem,i have no problem,i have no problem,i have no problem,i have no problem,i have no problem,i have no problem,i have no problem,i have no problem,...,i have no problem,i have no problem,i have no problem,i have no problem,i have no problem,i have no problem,i have no problem,i have no problem,i have no problem,i have no problem
2,I have no problems with it,I have no problems with it,I have no problems with it,I have no problems with it,I have no problems with it,I have no problems with it,I have no problems with it,I have no problems with it,I have no problems with it,I have no problems with it,...,I have no problems with it,I have no problems with it,I have no problems with it,I have no problems with it,I have no problems with it,I have no problems with it,I have no problems with it,I have no problems with it,I have no problems with it,"Yes, I have no problems with it"
3,i dont have problem in my canvas,i can easily login my canvas app in my mobile app,I can access my Canvas account using the avail...,i dont have problem in internet connection,i dont have problem with it,i dont have problem with it,i dont have problem with it,i dont have problem with it,i d

In [4]:
lem_df = pd.read_csv('lemmatized_opinion.csv')
lem_df = lem_df.iloc[0:, 0:-1] # Remove last columns
lem_df.head()

,I can easily log-in and log-out my Canvas account. (Please write your views relating your answer above.),I can access my course using the mobile Canvas app. (Please write your views about this relating to your answer above.),I can access my Canvas account using the available computers in JRU. (Please write your views about this relating to your answer above.),I have no issues accessing my Canvas account using the wired internet connection in the computer laboratories. (Please write your views about this relating to your answer above.),I can easily get help when I have problems with my Canvas account. (Please write your views about this relating to your answer above.),I can access my Canvas account outside JRU. (Please write your views about this relating to your answer above.),I can clearly understand the lessons on my Canvas modules. (Please write your views about this relating to your answer above.),I recognize that the modules in my course are well organized. (Please write your views about this relating to your answer above.),I can easily manage the navigation of the courses through the desktop version. .1,I can easily manage the navigation of the courses through the mobile app. (Please write your views about this relating to your answer above.),...,I understand the lessons better with the help of course materials in Canvas. (Please write your views about this relating to your answer above.),Canvas course materials make me interested. (Please write your views about this relating to your answer above.),"The course materials on Canvas helped me prepare for: Face to face discussions, quizzes, major exams, assignments, forums, etc. (Please write your views about this relating to your answer above.)",My instructor knows how to use Canvas. (Please write your views about this relating to your answer above.),My instructor demonstrated mastery of the subject matter. (Please write your views about this relating to your answer above.),My instructor encourages me to use Canvas. (Please write your views about this relating to your answer above.),"Canvas has made it easy for me to: communicate with my instructor, receive class announcements, share ideas on group discussions","I can answer the assessment materials (quiz, exam) posted in Canvas. (Please write your views about this relating to your answer above.)","I find the course materials on Canvas aligned with the: quizzes, major exams, assignments, forums, etc.) (Please write your views about this relating to your answer above.)",I have been give enough time to accomplish the activities on Canvas. (Please write your views about this relating to your answer above.)
0,neutral,neutral,neutral,neutral,neutral,neutral,neutral,neutral,neutral,neutral,...,neutral,neutral,neutral,neutral,neutral,neutral,neutral,neutral,neutral,neutral
1,problems log,problem,problem,problem,problem,problem,problem,problem,problem,problem,...,problem,problem,problem,problem,problem,problem,problem,problem,problem,problem
2,problems,problems,problems,problems,problems,problems,problems,problems,problems,problems,...,problems,problems,problems,problems,problems,problems,problems,problems,problems,yes problems
3,dont problem canvas,easily login canvas app mobile app,access canvas account using available computer...,dont problem internet connection,dont problem,dont problem,dont problem,dont problem,dont problem,dont problem,...,dont problem,dont problem,dont problem,dont problem,dont problem,dont problem,dont problem,dont problem,dont problem,dont problem
4,problems,access course,problem accessing canvas,dont issue,problem,problem,understand,easy recognize,manage,manage,...,understand lesson,im interested,course materials helped,yes teach us,yes demonstrate,encourage,communicate easy,problem,problem,problem


In [5]:
from nltk.corpus import stopwords
stoplist = stopwords.words('english') + ['though']

In [6]:
from sklearn.feature_extraction.text import CountVectorizer
from pprint import pprint

c_vec = CountVectorizer(stop_words=stoplist, ngram_range=(3,5))
comp = []
col_range = len(test_df.columns) # number of columns

for i in range(0,col_range):
    #col = test_df.columns[i] # The Unlemmatized Opinion
    col = lem_df.columns[i] # The Lemmatized Opinion
    
    ngrams = c_vec.fit_transform(test_df[col])
    vocab = c_vec.vocabulary_
    vocab = vocab.keys()
    
    #Create dataframe to store different lengths of ngrams
    ## Note Cannot properly store Ngrams due to uneven row count, so every iteration the previous column is dropped
    df = pd.DataFrame(vocab, columns = ['ngram'])
    
    df['scores'] = df['ngram'].apply(lambda x: sid.polarity_scores(x)) ## Get polarity score of every Column and Row
    compound = df['scores'].apply(lambda score_dict: score_dict['compound']) ## Extract the compound from the results
    
    ave = np.average(compound)# Get the mean compound of each columns
    comp.append(ave) # Save mean and append to list
    #print("Length of Ngram in column:",i+1,len(vocab))

    
        
    #print("Sentiment Score per row: ",comp) ##
    print("Mean Sentiment Score for Column",i+1,":",ave) ## 


Mean Sentiment Score for Column 1 : 0.09742626262626262
Mean Sentiment Score for Column 2 : 0.08671189839572192
Mean Sentiment Score for Column 3 : 0.08418537666174299
Mean Sentiment Score for Column 4 : 0.02600934579439252
Mean Sentiment Score for Column 5 : 0.09872561174551386
Mean Sentiment Score for Column 6 : 0.06834012345679014
Mean Sentiment Score for Column 7 : 0.1551563535911602
Mean Sentiment Score for Column 8 : 0.20349373297002724
Mean Sentiment Score for Column 9 : 0.24629525222551926
Mean Sentiment Score for Column 10 : 0.11851658653846155
Mean Sentiment Score for Column 11 : 0.085259793814433
Mean Sentiment Score for Column 12 : 0.27980666666666665
Mean Sentiment Score for Column 13 : 0.29479348370927316
Mean Sentiment Score for Column 14 : 0.17662116564417174
Mean Sentiment Score for Column 15 : 0.11872903225806453
Mean Sentiment Score for Column 16 : 0.24922847222222222
Mean Sentiment Score for Column 17 : 0.170012962962963
Mean Sentiment Score for Column 18 : 0.149983

In [7]:
comp[0]

0.09742626262626262

In [8]:
df # Sample ngram

,ngram,scores
0,give week deadline,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound..."
1,week deadline activity,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound..."
2,give week deadline activity,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound..."
3,deadline every activities,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound..."
4,every activities able,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound..."
...,...,...
456,hard finish comes due date,"{'neg': 0.259, 'neu': 0.741, 'pos': 0.0, 'comp..."
457,time extended needed,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound..."
458,yes sometimes extends,"{'neg': 0.0, 'neu': 0.192, 'pos': 0.808, 'comp..."
459,sometimes extends deadline,"{'neg': 0.0, 'neu': 0.571, 'pos': 0.429, 'comp..."


In [9]:
'''
Compound VADER scores for analyzing sentiment
The compound score is computed by summing the valence scores of each word in the lexicon, 
adjusted according to the rules, 
and then normalized to be between -1 (most extreme negative) and +1 (most extreme positive).
'''

print(comp)

[0.09742626262626262, 0.08671189839572192, 0.08418537666174299, 0.02600934579439252, 0.09872561174551386, 0.06834012345679014, 0.1551563535911602, 0.20349373297002724, 0.24629525222551926, 0.11851658653846155, 0.085259793814433, 0.27980666666666665, 0.29479348370927316, 0.17662116564417174, 0.11872903225806453, 0.24922847222222222, 0.170012962962963, 0.14998398510242084, 0.11023876923076922, 0.07220956284153006, 0.11779370932754882]


### NGRAM CORRELATION

In [10]:
compound = pd.DataFrame(comp, columns=["Lemmatized and Ngram"])
compound.to_csv("ngram_sentiment_score.csv", index=False)

In [19]:
survey_df = pd.read_csv('survey_numerical_value.csv')
survey_df = survey_df.drop(survey_df.columns[0], axis=1)
survey_df = survey_df.drop(survey_df.index[[14,15,16,21,22,25,26]])
survey_df["Sentiment Score"] = comp



In [20]:
df1 = pd.read_csv('survey_numerical_value.csv')
df1 = df1.drop(df1.columns[0], axis=1)

merged = survey_df.combine_first(df1)

order = ['index','Strongly Agree', 'Agree', 'Disagree','Strongly Disagree','Numerical Value', 'Sentiment Score']
merged = merged.reindex(columns=order)
merged

,index,Strongly Agree,Agree,Disagree,Strongly Disagree,Numerical Value,Sentiment Score
0,I can easily log-in and log-out my Canvas acco...,124.0,12.0,1.0,3.0,3.84,0.097426
1,I can access my course using the mobile Canvas...,112.0,17.0,5.0,6.0,3.68,0.086712
2,I can access my Canvas account using the avail...,124.0,11.0,2.0,3.0,3.83,0.084185
3,I can easily get help when I have problems wit...,103.0,18.0,8.0,11.0,3.52,0.026009
4,I have no issues accessing my Canvas account u...,114.0,12.0,8.0,6.0,3.67,0.098726
5,I can access my Canvas account outside JRU.,122.0,11.0,6.0,1.0,3.81,0.068340
6,I can clearly understand the lessons on my Can...,105.0,21.0,9.0,5.0,3.61,0.155156
7,I recognize that the modules in my course are ...,116.0,19.0,3.0,2.0,3.78,0.203494
8,I can easily manage the navigation of the cour...,121.0,16.0,0.0,3.0,3.82,0.246295
9,I can easily manage the navigation of the cour...,103.0,18.0,10.0,9.0,3.54,0.118517


In [21]:
ngram_df = merged.drop(merged.columns[[1,2,3,4]],axis=1)
ngram_df

,index,Numerical Value,Sentiment Score
0,I can easily log-in and log-out my Canvas acco...,3.84,0.097426
1,I can access my course using the mobile Canvas...,3.68,0.086712
2,I can access my Canvas account using the avail...,3.83,0.084185
3,I can easily get help when I have problems wit...,3.52,0.026009
4,I have no issues accessing my Canvas account u...,3.67,0.098726
5,I can access my Canvas account outside JRU.,3.81,0.068340
6,I can clearly understand the lessons on my Can...,3.61,0.155156
7,I recognize that the modules in my course are ...,3.78,0.203494
8,I can easily manage the navigation of the cour...,3.82,0.246295
9,I can easily manage the navigation of the cour...,3.54,0.118517


In [22]:
#Aggregate Same Likert and Opinions first
# Manually fix inequal rows
'''
Numerical_mean  = corr.iloc[0:, 1].mean()
Sentiment_mean  = corr.iloc[0:, 2].mean()
means = [Numerical_mean,Sentiment_mean]
corr.loc[21] = ['Average'] + means
'''
                        ###Row  #Col
numerical_mean_first  = ngram_df.iloc[13:17, 1].mean()
sentiment_mean_first = ngram_df.iloc[13,2]

numerical_mean_sec  = ngram_df.iloc[20:23, 1].mean()
sentiment_mean_sec = ngram_df.iloc[20,2]

numerical_mean_third  = ngram_df.iloc[24:27, 1].mean()
sentiment_mean_third = ngram_df.iloc[24,2]

means1 = [numerical_mean_first,sentiment_mean_first]
ngram_df.loc[29] = ['1st aggregation'] + means1

means2 = [numerical_mean_sec,sentiment_mean_sec]
ngram_df.loc[30] = ['2nd aggregation'] + means2

means3 = [numerical_mean_third,sentiment_mean_third]
ngram_df.loc[31] = ['3rd aggregation'] + means3

Numerical_mean  = ngram_df.iloc[0:, 1].mean()
Sentiment_mean  = ngram_df.iloc[0:, 2].mean()
means = [Numerical_mean,Sentiment_mean]
ngram_df.loc[32] = ['Average'] + means

fixed_df = ngram_df.drop([13,14,15,16,20,21,22,24,25,26])

fixed_df.reset_index(drop=True)
fixed_df.to_csv("fixed_ngram.csv", index=False)
fixed_df

,index,Numerical Value,Sentiment Score
0,I can easily log-in and log-out my Canvas acco...,3.840000,0.097426
1,I can access my course using the mobile Canvas...,3.680000,0.086712
2,I can access my Canvas account using the avail...,3.830000,0.084185
3,I can easily get help when I have problems wit...,3.520000,0.026009
4,I have no issues accessing my Canvas account u...,3.670000,0.098726
5,I can access my Canvas account outside JRU.,3.810000,0.068340
6,I can clearly understand the lessons on my Can...,3.610000,0.155156
7,I recognize that the modules in my course are ...,3.780000,0.203494
8,I can easily manage the navigation of the cour...,3.820000,0.246295
9,I can easily manage the navigation of the cour...,3.540000,0.118517


In [23]:
x_var = fixed_df.iloc[0:-1, 1]
y_var = fixed_df.iloc[0:-1, 2]
y_var

0     0.097426
1     0.086712
2     0.084185
3     0.026009
4     0.098726
5     0.068340
6     0.155156
7     0.203494
8     0.246295
9     0.118517
10    0.085260
11    0.279807
12    0.294793
17    0.118729
18    0.249228
19    0.170013
23    0.110239
27    0.117794
29    0.176621
30    0.149984
31    0.072210
Name: Sentiment Score, dtype: float64

In [25]:
import scipy

pearson_r = scipy.stats.pearsonr(x_var, y_var) #Pearson Correlation Analysis
pearson_r = np.reshape(pearson_r, (1, 2))
corr_analysis = pd.DataFrame(pearson_r, columns=["R", "p-Value"])
corr_analysis

,R,p-Value
0,0.124063,0.592103
